In [7]:
import os, random
import chess
import chess.pgn
import numpy as np

In [8]:
# let's start with super simple bot that just plays random legal moves
class super_simple_bot:
    def make_a_random_move(__self__, board):
        legal_moves = []
        for move in board.legal_moves:
            legal_moves.append(move)
        board.push(legal_moves[random.randrange(len(legal_moves))])
chessbot = super_simple_bot()
board = chess.Board()
chessbot.make_a_random_move(board)


Well, what if we want the bot to play like me? I have a whole bunch of games played on chess.com and I can download the data in month batches.

Let's do something else really simple. If I've played games with the same move order before, have the bot play my most played move for that move order.

We could do this without considering move order and just evaluate each position, but that's computationally quite intensive, and in most games played by players without titles you rarely transpose positions.

In [51]:
class super_simple_bot:

    def __init__(self, username, games_dir):
        self.username = username
        self.game_filenames = []
        for filename in os.listdir(games_dir):
            self.game_filenames.append(games_dir+filename)

    def find_a_random_move(self, board):
        legal_moves = []
        for move in board.legal_moves:
            legal_moves.append(move)
        return legal_moves[random.randrange(len(legal_moves))]

    def playing_as(self, game):
        if game.headers["Black"] == self.username:
            return chess.BLACK
        else:
            return chess.WHITE

    def find_my_most_played_move(self, position):
        position_fen = position.fen()
        candidate_move_dict = {}

        for file in self.game_filenames:
            pgn = open(file)
            game = chess.pgn.read_game(pgn)
            while game is not None:
                my_colour = self.playing_as(game)
                board = game.board()
                for move in game.mainline_moves():
                    if board.turn == my_colour:
                        if board.fen() == position_fen:
                            if move in candidate_move_dict:
                                candidate_move_dict[move] += 1
                            else:
                                candidate_move_dict[move] = 1
                    board.push(move)
                game = chess.pgn.read_game(pgn)
            pgn.close()
        if candidate_move_dict:
            chosen_move = max(candidate_move_dict, key = candidate_move_dict.get)
            return chosen_move
        else:
            return None


In [52]:
chessbot = super_simple_bot("dylanjsw", "played_games/")
board = chess.Board()
move = chessbot.find_my_most_played_move(board)
while move is not None:
    board.push(move)
    move = chessbot.find_my_most_played_move(board)
board

Terrific - now what if we reach a position where I've never played a move in that position before? What's the kind of move I *would* have played? This is where we need to be able to predict my moves based on my previous moves.

There is a lot of documentation on creating simple chess engines, but there isn't a lot of documentation on creating chess AIs based on previously played games. The example I am following here is from [Ethan Mai](https://towardsdatascience.com/machine-learning-how-i-coded-my-own-python-chess-engine-c4c81bcb2a9a).

The first step is to create the dataset to train on. We need to create a data set that records the board state, the move I played, and the legal moves I didn't play. I am leaning heavily on Ethan's work here.

In [11]:
class smart_bot(super_simple_bot):

    def label_my_played_games(self):
        for file in self.game_filenames:
            pgn = open(file)
            game = chess.pgn.read_game(pgn)
            while game is not None:
                for move in game.mainline_moves():
                    # make entries in my_moves and other_moves for the board state
                game = chess.pgn.read_game(pgn)
            pgn.close()

chessbot = smart_bot("dylanjsw", "played_games/")
chessbot.label_my_played_games()